# Exhaustive search - Flowshop problem



### Table of content
- [Brute force](#Brute-force)
- [Heuristics](#Heuristics)
- [Branch & Bound](#Branch-&-Bound)
- [Tests](#Tests)
   


### References
- [Finding an Optimal Sequence in the Flowshop Scheduling Using Johnson’s Algorithm](https://ijiset.com/vol2/v2s1/IJISET_V2_I1_50.pdf)
- [Benchmarks for Basic Scheduling Problems](http://mistic.heig-vd.ch/taillard/articles.dir/Taillard1993EJOR.pdf)

In [ ]:
import numpy as np
import matplotlib as plt
import itertools
import time
import math

# Heuristics

## NEH Algorithm

In [ ]:
def order_jobs_in_descending_order_of_total_completion_time(processing_times):
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

In [ ]:
def insertion(sequence, position, value):
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq

In [ ]:
def neh_algorithm(processing_times):
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    # Define the initial order
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if evaluate_sequence([J1, J2], processing_times) < evaluate_sequence([J2, J1], processing_times) else [J2, J1]
    del ordered_sequence[:2]
    # Add remaining jobs
    for job in ordered_sequence:
        Cmax = float('inf')
        best_sequence = []
        for i in range(len(sequence)+1):
            new_sequence = insertion(sequence, i, job)
            Cmax_eval = evaluate_sequence(new_sequence, processing_times)
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval
                best_sequence = new_sequence
        sequence = best_sequence
    return sequence, Cmax

# Branch & Bound

- Branch & Bound.
- Branch & Bound pure.

In [ ]:
# Define the Node structure of the seach tree that we will be using
class Node:
    def __init__(self, jobs, remaining_jobs, parent=None, lower_bound=1e100):
        self.jobs = jobs
        self.remaining_jobs = remaining_jobs
        self.parent = parent
        self.lower_bound = lower_bound
    def __str__(self):
        return f"Node(jobs={self.jobs}, remaining_jobs={self.remaining_jobs}, lower_bound={self.lower_bound})"
    def __lt__(self, other):
        # Define less than comparison for the priority queue
        return self.lower_bound < other.lower_bound

    def __eq__(self, other):
        # Define equality for the priority queue
        return self.lower_bound == other.lower_bound

In [ ]:
def evaluate_sequence(sequence, processing_times):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)

    # Check if the sequence is empty
    if num_jobs == 0:
        # Return a default value (you may choose 0 or another suitable value)
        return 0

    completion_times = np.zeros((num_jobs, num_machines))

    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]

    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]

    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]


In [ ]:
import numpy as np

def job_completion_times(jobs_matrix, sequence):
    num_jobs, num_machines = jobs_matrix.shape
    completion_matrix = np.zeros((len(sequence), num_machines))

    for i in range(len(sequence)):
        for j in range(num_machines):
            if i == 0 and j == 0:
                completion_matrix[0, j] = jobs_matrix[sequence[i], j]
            elif i == 0:
                completion_matrix[0, j] = jobs_matrix[sequence[i], j] + completion_matrix[0, j - 1]
            elif j == 0:
                completion_matrix[i, 0] = jobs_matrix[sequence[i], j] + completion_matrix[i - 1, 0]
            else:
                completion_matrix[i, j] = jobs_matrix[sequence[i], j] + max(completion_matrix[i - 1, j], completion_matrix[i, j - 1])

    return completion_matrix

# Example usage:
jobs_matrix = np.array([
    [5, 6, 4],
    [8, 7, 8],
    [7, 2, 7],
    [3, 5, 9]
])

job_sequence = [3, 0]
result = job_completion_times(jobs_matrix, job_sequence)
print(result)


[[ 3.  8. 17.]
 [ 8. 14. 21.]]


In [ ]:
import numpy as np

matrix = np.array([
    [53, 19, 99, 62, 88, 93, 34, 72, 42, 65, 39, 79, 9, 26, 72, 29, 36, 48, 57, 95],
    [93, 79, 88, 77, 94, 39, 74, 46, 17, 30, 62, 77, 43, 98, 48, 14, 45, 25, 98, 30],
    [90, 92, 35, 13, 75, 55, 80, 67, 3, 93, 54, 67, 25, 77, 38, 98, 96, 20, 15, 36],
    [65, 97, 27, 25, 61, 24, 97, 61, 75, 92, 73, 21, 29, 3, 96, 51, 26, 44, 56, 31],
    [64, 38, 44, 46, 66, 31, 48, 27, 82, 51, 90, 63, 85, 36, 69, 67, 81, 18, 81, 72]
])



matrix = np.transpose(matrix)

column_sums = [sum(col) for col in zip(*matrix)]


lign_sums = row_sums = [sum(row) for row in matrix]


def fist_lower_baund(processing_times, column_sums, lign_sums):
    lb1 = column_sums[0] + min(lign_sums[1:])
    lb2 = column_sums[1] + min(lign_sums[:1] + lign_sums[2:])
    lb3 = column_sums[2] + min(lign_sums[:2] + lign_sums[3:])
    lb4 = column_sums[3] + min(lign_sums[:3] + lign_sums[4:])
    lb5 = column_sums[4] + min(lign_sums[:4] + lign_sums[5:])

    return max(lb1, lb2, lb3, lb4, lb5)


def first_LBA (processing_times,jobs,column_sums) :
    first_job=jobs[0]
    matrix =  processing_times[:, 1:]
    matrix = np.delete(matrix, jobs[0], axis=0)
    row_sums = np.sum(matrix, axis=1)
    lb1=column_sums[0] +np.min(row_sums)
    matrix =  matrix[:, 1:]
    row_sums = np.sum(matrix, axis=1)
    lb2=column_sums[1]+processing_times[first_job,0]+np.min(row_sums)
    matrix =  matrix[:, 1:]
    row_sums = np.sum(matrix, axis=1)
    lb3=processing_times[first_job,0]+processing_times[first_job,1]+column_sums[2]+np.min(row_sums)
    matrix =  matrix[:, 1:]
    row_sums = np.sum(matrix, axis=1)
    lb4=processing_times[first_job,0]+processing_times[first_job,1]+processing_times[first_job,2]+column_sums[3]+np.min(row_sums)

    lb5=processing_times[first_job,0]+processing_times[first_job,1]+processing_times[first_job,2]+processing_times[first_job,3]+column_sums[4]

    return max(lb1,lb2,lb3,lb4,lb5)

def LBA (processing_times,jobs,column_sums) :

    matrix =  processing_times[:, 1:]

    # Create a matrix without the jobs exist in the jobs list
    mask = np.ones(len(matrix), dtype=bool)
    mask[jobs] = False
    matrix = matrix[mask]
    #-------------------------------------------------------
    #
    row_sums = np.sum(matrix, axis=1)
    M =job_completion_times(processing_times, jobs)
    #---------------------lb1
    lb1=column_sums[0] +np.min(row_sums)


    #---------------------lb2

    lb2=M[len(jobs)-1,1]+np.sum(matrix[:, 0])
    matrix =  matrix[:, 1:]
    row_sums = np.sum(matrix, axis=1)
    lb2=lb2+np.min(row_sums)
    #---------------------lb3

    lb3=M[len(jobs)-1,2]+np.sum(matrix[:, 0])
    matrix =  matrix[:, 1:]
    row_sums = np.sum(matrix, axis=1)
    lb3=lb3+np.min(row_sums)
    #---------------------lb4
    lb4=M[len(jobs)-1,3]+np.sum(matrix[:, 0])
    matrix =  matrix[:, 1:]
    row_sums = np.sum(matrix, axis=1)
    lb4=lb4+np.min(row_sums)
    #---------------------lb5
    lb5=M[len(jobs)-1,4]+np.sum(matrix[:, 0])



    return max(lb1,lb2,lb3,lb4,lb5)
print(first_LBA (matrix,[19],column_sums))
LBA(matrix,[19],column_sums)



1354


1354.0

## Branch & Bound

In [ ]:
from typing import List

def branch_and_bound(processing_times, initial_solution, initial_cost):
    column_sums = [sum(col) for col in zip(*processing_times)]
    jobs, machines = processing_times.shape
    root_node = Node([], set(range(jobs)))
    best_solution = initial_solution.copy()
    best_solution_cost = initial_cost

    def dfs(node):

        nonlocal best_solution, best_solution_cost
        if ((len(node.jobs)!=0 and node.lower_bound < best_solution_cost) or (len(node.jobs)==0)):
            # Calculate the lower bounds for all remaining jobs
            lower_bounds = []
            for job in node.remaining_jobs:
                child_jobs = node.jobs + [job]
                child_remaining_jobs = node.remaining_jobs - {job}

                # Calculate the lower bound for the child node
                ev = evaluate_sequence(child_jobs, processing_times)
                if not child_remaining_jobs:
                    child_lower_bound = ev
                else:
                    child_lower_bound = LBA(processing_times, child_jobs, column_sums)

                if not child_remaining_jobs and job == max(node.remaining_jobs):
                    if child_lower_bound < best_solution_cost:
                        best_solution = child_jobs
                        best_solution_cost = child_lower_bound
                        print('new best solution', best_solution)
                        print('new best cost', best_solution_cost)
                    continue

                # If the child node is not a leaf, continue the depth-first search
                if child_lower_bound < best_solution_cost:
                    child_node = Node(child_jobs, child_remaining_jobs, parent=node, lower_bound=child_lower_bound)
                    lower_bounds.append(child_node)

            # Sort the child nodes based on their lower bounds
            sorted_nodes = sorted(lower_bounds, key=lambda x: x.lower_bound)
            for sorted_node in sorted_nodes:
                dfs(sorted_node)

    # Start the depth-first search with the root node
    dfs_stack = [root_node]
    i = 0
    while dfs_stack:
        current_node = dfs_stack.pop()
        dfs(current_node)
        i += 1

    return best_solution, best_solution_cost, i


## Common Instance

In [ ]:
instance_common = np.array([
    [71, 79, 85, 82, 83],
    [84, 71, 66, 68, 81],
    [78, 81, 75, 72, 87],
    [78, 75, 66, 72, 88],
    [72, 88, 83, 85, 88],
    [86, 88, 79, 82, 78],
    [75, 66, 86, 78, 78],
    [80, 79, 66, 83, 78],
    [73, 73, 67, 77, 71],
    [80, 77, 83, 78, 67],
])

In [ ]:
instance_common.shape

(10, 5)

In [ ]:
initial_solution = [i for i in range(10)]
initial_cost = evaluate_sequence(initial_solution, instance_common)

start_time = time.time()
best_solution, best_cost,i= branch_and_bound(instance_common, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'Results of Branch & Bound:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'No. Nodes visited is {i}.')
print(f'Elapsed time of {elapsed_time} seconds.')

new best solution [1, 8, 3, 6, 2, 0, 4, 5, 7, 9]
new best cost 1115.0
new best solution [1, 8, 3, 6, 0, 2, 4, 5, 7, 9]
new best cost 1110.0
new best solution [1, 3, 6, 0, 2, 7, 8, 4, 5, 9]
new best cost 1109.0
new best solution [1, 3, 6, 0, 2, 7, 4, 8, 5, 9]
new best cost 1108.0
new best solution [8, 3, 6, 2, 7, 1, 0, 4, 5, 9]
new best cost 1107.0
new best solution [8, 3, 6, 2, 1, 0, 4, 5, 7, 9]
new best cost 1105.0
new best solution [3, 6, 2, 0, 1, 8, 4, 5, 7, 9]
new best cost 1104.0
new best solution [3, 2, 7, 6, 8, 0, 1, 4, 5, 9]
new best cost 1102.0
Results of Branch & Bound:
Best sequence is [3, 2, 7, 6, 8, 0, 1, 4, 5, 9] with a makespan of 1102.0.
No. Nodes visited is 1.
Elapsed time of 0.8088369369506836 seconds.


In [ ]:
import numpy as np

matrix = np.array([
    [53, 19, 99, 62, 88, 93, 34, 72, 42, 65, 39, 79, 9, 26, 72, 29, 36, 48, 57, 95],
    [93, 79, 88, 77, 94, 39, 74, 46, 17, 30, 62, 77, 43, 98, 48, 14, 45, 25, 98, 30],
    [90, 92, 35, 13, 75, 55, 80, 67, 3, 93, 54, 67, 25, 77, 38, 98, 96, 20, 15, 36],
    [65, 97, 27, 25, 61, 24, 97, 61, 75, 92, 73, 21, 29, 3, 96, 51, 26, 44, 56, 31],
    [64, 38, 44, 46, 66, 31, 48, 27, 82, 51, 90, 63, 85, 36, 69, 67, 81, 18, 81, 72]
])

matrix = np.transpose(matrix)

neh_solution =neh_algorithm(matrix)

print(neh_solution[0])



[12, 8, 15, 10, 14, 6, 9, 18, 16, 0, 4, 1, 11, 7, 13, 2, 19, 5, 3, 17]


In [ ]:
initial_solution = neh_solution[0]
initial_cost = neh_solution[1]

start_time = time.time()
best_solution, best_cost, i = branch_and_bound(matrix, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'Results of Branch & Bound:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'No. Nodes visited is {i}.')
print(f'Elapsed time of {elapsed_time} seconds.')

new best solution [12, 8, 15, 13, 3, 19, 10, 14, 16, 6, 0, 18, 9, 1, 11, 4, 7, 2, 5, 17]
new best cost 1337.0
new best solution [12, 8, 15, 13, 3, 19, 10, 14, 16, 6, 18, 9, 11, 1, 0, 4, 7, 2, 5, 17]
new best cost 1335.0
new best solution [12, 8, 15, 13, 3, 14, 16, 10, 6, 19, 18, 9, 0, 1, 11, 4, 7, 2, 5, 17]
new best cost 1334.0
new best solution [12, 8, 15, 13, 3, 14, 10, 9, 16, 18, 6, 19, 1, 0, 4, 7, 11, 17, 2, 5]
new best cost 1333.0
new best solution [12, 8, 15, 13, 3, 14, 9, 16, 18, 6, 19, 10, 1, 0, 4, 7, 11, 17, 2, 5]
new best cost 1332.0
new best solution [12, 8, 15, 13, 3, 14, 9, 16, 18, 10, 6, 19, 1, 0, 4, 7, 11, 17, 2, 5]
new best cost 1329.0
new best solution [12, 8, 15, 13, 11, 17, 14, 10, 9, 16, 18, 6, 19, 1, 0, 4, 7, 2, 5, 3]
new best cost 1324.0
new best solution [12, 8, 15, 13, 11, 14, 10, 9, 16, 18, 1, 19, 6, 4, 0, 2, 7, 3, 17, 5]
new best cost 1323.0
new best solution [12, 8, 15, 13, 11, 14, 10, 9, 16, 18, 6, 19, 1, 4, 0, 2, 7, 3, 17, 5]
new best cost 1320.0
new best s